<a href="https://colab.research.google.com/github/gusti-amber/udemy_pytorch/blob/main/lecture/lec3_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 自動微分
Tensorは、requires_grad属性をTrueに設定することで計算過程が記録されるようになります。

In [1]:
import torch

x = torch.ones(2,3,requires_grad = True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


# 2. Tensorの演算と自動微分
requires_grad属性がTrueであれば、Tensorの加算、乗算、平均値などの演算でgrad_fnが記録される。

In [4]:
y = x+2
print(y)
print(y.grad_fn)

z = y*3
print(z)
print(z.grad_fn)

out = z.mean()
print(out)
print(out.grad_fn)

tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)
tensor([[9., 9., 9.],
        [9., 9., 9.]], grad_fn=<MulBackward0>)
tensor(9., grad_fn=<MeanBackward0>)


# 3. 勾配の計算
backwardメソッドは逆伝播により勾配を計算する。

その際に、記録されている演算と経路が使用されます。

この場合、bを逆伝播することで、aに対するbの勾配が計算される。
勾配はa.gradで取得できる。

In [7]:
a = torch.tensor([1.0], requires_grad=True)
b = a*2
b.backward()
print(a.grad)

tensor([2.])


より複雑な経路を持つ演算でも、backwardにより勾配を計算できる。

In [10]:
def calc(a):
    b = a*2 + 1
    c = b*b
    d = c/(c + 2)
    e = d.mean()
    return e

x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=True)

y = calc(x)
y.backward()
print(x.grad.tolist())

[0.06611569225788116, 0.01828990876674652, 0.007176725193858147]


xの各値付近の勾配を求めた。

この勾配が正しく計算できているか、xの書く値を微小変化させて確かめる。

In [11]:
delta = 0.001  #xの微小変化

x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=True)
y = calc(x).item()

x_1 = [1.0+delta, 2.0, 3.0]
x_1 = torch.tensor(x_1, requires_grad=True)
y_1 = calc(x_1).item()

x_2 = [1.0, 2.0+delta, 3.0]
x_2 = torch.tensor(x_2, requires_grad=True)
y_2 = calc(x_2).item()

x_3 = [1.0, 2.0, 3.0+delta]
x_3 = torch.tensor(x_3, requires_grad=True)
y_3 = calc(x_3).item()

# 勾配の計算
grad_1 = (y_1 - y) / delta
grad_2 = (y_2 - y) / delta
grad_3 = (y_3 - y) / delta

print(grad_1, grad_2, grad_3)

0.06604194641113281 0.018298625946044922 0.007152557373046875


xの微小変化を0.001という小さい値にしましたが、yの微小変化との割合はbackwardによる計算結果とほぼ同じになりました。

backwardにより正しく勾配を計算できていることが確認できました。